# 데이터 불러오기

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
base_path = './preprocessed data/'

train_err = pd.read_csv(base_path + 'new_train_err.csv', parse_dates=['time'])
train_quality = pd.read_csv(base_path + 'new_train_quality.csv', parse_dates=['time'])
train_problem = pd.read_csv(base_path + 'new_train_problem.csv', parse_dates=['time'])
test_err = pd.read_csv(base_path + 'new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(base_path + 'new_test_quality.csv', parse_dates=['time'])
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

# feature engineering

default = model_nm, fwver, errtype, counts
1. hour + multiple_model + multiple_fwver + errtype_num + errtype_weight + errcode_weight + quality_sum

In [163]:
df1 = train_err.copy()
df1

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
15367996,24999,2020-11-30 16:30:51,model_3,05.15.2138,15,1
15367997,24999,2020-11-30 17:26:25,model_3,05.15.2138,16,1
15367998,24999,2020-11-30 17:26:25,model_3,05.15.2138,4,0
15367999,24999,2020-11-30 17:26:31,model_3,05.15.2138,4,0


In [164]:
# hour와 counts column 추가
df1['hour'] = df1.time.dt.hour
df1['counts'] = 1
df1

,user_id,time,model_nm,fwver,errtype,errcode,hour,counts
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1,2,1
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1,3,1
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1,3,1
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1,5,1
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0,5,1
...,...,...,...,...,...,...,...,...
15367996,24999,2020-11-30 16:30:51,model_3,05.15.2138,15,1,16,1
15367997,24999,2020-11-30 17:26:25,model_3,05.15.2138,16,1,17,1
15367998,24999,2020-11-30 17:26:25,model_3,05.15.2138,4,0,17,1
15367999,24999,2020-11-30 17:26:31,model_3,05.15.2138,4,0,17,1


In [165]:
# errtype, errcode에 대한 가중치 column추가 (errtype_weight, errcode_weight)

# 1. errtype_weight
errtype_df  = train_err[['user_id', 'errtype']]
errtype_df = errtype_df.drop_duplicates(subset=None, keep='first')
errtype_df = errtype_df.groupby('errtype')['user_id'].unique().to_frame().reset_index()
errtype_df['counts'] = 0
errtype_df['errtype_weight'] = 0

for i in range(len(errtype_df.index)):
    errtype_df['counts'][i] = len(list(train_problem[train_problem['user_id'].isin(errtype_df.user_id[i])].user_id.unique()))
    
for i in range(len(errtype_df.index)):
        errtype_df['errtype_weight'][i] = round(errtype_df['counts'][i]/len(errtype_df.user_id[i]), 2)
errtype_df.drop(columns=['user_id', 'counts'], inplace=True)

errtype_df.head()

,errtype,errtype_weight
0,1,0.25
1,2,0.43
2,3,0.28
3,4,0.33
4,5,0.37


In [166]:
# 2. errcode_weight
errcode_df  = train_err[['user_id', 'errcode']]
errcode_df = errcode_df.drop_duplicates(subset=None, keep='first')
errcode_df = errcode_df.groupby('errcode')['user_id'].unique().to_frame().reset_index()
errcode_df['counts'] = 0
errcode_df['errcode_weight'] = 0

for i in range(len(errcode_df.index)):
    errcode_df['counts'][i] = len(list(train_problem[train_problem['user_id'].isin(errcode_df.user_id[i])].user_id.unique()))
    
for i in range(len(errcode_df.index)):
        errcode_df['errcode_weight'][i] = round(errcode_df['counts'][i]/len(errcode_df.user_id[i]), 2)
errcode_df.drop(columns=['user_id', 'counts'], inplace=True)

errcode_df.head()

,errcode,errcode_weight
0,-269,0.64
1,-270,0.36
2,0,0.34
3,0001,0.50
4,1,0.33


In [167]:
df1 = pd.merge(df1, errtype_df, how='left', on='errtype')
df1 = pd.merge(df1, errcode_df, how='left', on='errcode')
df1.head(10)

,user_id,time,model_nm,fwver,errtype,errcode,hour,counts,errtype_weight,errcode_weight
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1,2,1,0.34,0.33
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1,3,1,0.34,0.33
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1,3,1,0.34,0.33
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1,5,1,0.34,0.33
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0,5,1,0.33,0.34
5,10000,2020-11-01 05:05:16,model_3,05.15.2138,26,1,5,1,0.36,0.33
6,10000,2020-11-01 05:05:22,model_3,05.15.2138,4,0,5,1,0.33,0.34
7,10000,2020-11-01 06:09:03,model_3,05.15.2138,15,1,6,1,0.34,0.33
8,10000,2020-11-01 17:47:54,model_3,05.15.2138,16,1,17,1,0.34,0.33
9,10000,2020-11-01 17:47:56,model_3,05.15.2138,4,0,17,1,0.33,0.34


In [168]:
# multiple_model
model_df =  train_err[['user_id', 'model_nm']]
model_df = model_df.drop_duplicates(subset=None, keep='first')

model_num = model_df.user_id.value_counts(sort=False)
user_has_multiple_models = model_num > 1

user_has_multiple_models = user_has_multiple_models.astype(int)
user_has_multiple_models = user_has_multiple_models.to_frame(name='multiple_model')
user_has_multiple_models.head(7)

,multiple_model
10000,0
10001,0
10002,0
10003,0
10004,0
10005,0
10006,1


In [155]:
# multiple_fwver
fwver_df =  train_err[['user_id', 'fwver']]
fwver_df = fwver_df.drop_duplicates(subset=None, keep='first')

fwver_num = fwver_df.user_id.value_counts(sort=False)
user_has_multiple_fwvers = fwver_num > 1

user_has_multiple_fwvers = user_has_multiple_fwvers.astype(int)
user_has_multiple_fwvers = user_has_multiple_fwvers.to_frame(name='multiple_fwver')
user_has_multiple_fwvers.head(7)

,multiple_fwver
10000,0
10001,1
10002,0
10003,1
10004,1
10005,1
10006,1


In [169]:
# errtype_num
errtype_df = train_err[['user_id', 'errtype']]
errtype_df = errtype_df.user_id.value_counts(sort=False).to_frame(name='errtype_num')
errtype_df.head()

,errtype_num
10000,316
10001,2345
10002,305
10003,297
10004,777


In [170]:
# quality_sum
quality_df = pd.DataFrame({'user_id': np.arange(10000, 25000)})

for i in train_quality.columns[3:]:
    quality_group_df = train_quality.groupby('user_id')[i].sum().to_frame().reset_index()
    quality_df = pd.merge(quality_df, quality_group_df, how='left', on='user_id')
    quality_df = quality_df.fillna(0)
    
# 모두 0값인 quality_3, quality_4 drop
quality_df.set_index(quality_df['user_id'], inplace=True)
quality_df = quality_df.drop(columns=['user_id', 'quality_3', 'quality_4'])

quality_df.head()

,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,,,,,,,,,,,
10000,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,1.0,-1.0,0.0,30.0,45.0,96.0,0.0,2.0,95.0,-1.0,0.0
10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,-1.0,-1.0,-1.0,3.0,86.0,174.0,0.0,0.0,14.0,-1.0,0.0


Train set 만들기

In [171]:
# 1. one-hot-encoding
df1.drop(columns=['time', 'errcode'], inplace=True)
df1 = pd.get_dummies(df1, columns=['model_nm', 'fwver', 'errtype'])
df1

,user_id,hour,counts,errtype_weight,errcode_weight,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,10000,2,1,0.34,0.33,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10000,3,1,0.34,0.33,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10000,3,1,0.34,0.33,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,10000,5,1,0.34,0.33,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,5,1,0.33,0.34,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367996,24999,16,1,0.34,0.33,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15367997,24999,17,1,0.34,0.33,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15367998,24999,17,1,0.33,0.34,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15367999,24999,17,1,0.33,0.34,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
# 2. errtype_weight, errcode_weight를 user_id별로 다시 가중치 적용

# 2-1. errtype_weight
errtype_weight_df = df1[['user_id', 'errtype_weight']]
errtype_weight_df = errtype_weight_df.groupby('user_id')['errtype_weight'].sum().to_frame(name='errtype_weight')
errtype_weight_df.errtype_weight = errtype_weight_df.errtype_weight / total_sum
errtype_weight_df

,errtype_weight
user_id,
10000,1.864538e-05
10001,1.549039e-04
10002,1.786448e-05
10003,1.858874e-05
10004,4.892524e-05
...,...
24995,1.217510e-05
24996,2.969119e-07
24997,5.098132e-05


In [174]:
# 2-2. errcode_weight
errcode_weight_df = df1[['user_id', 'errcode_weight']]
errcode_weight_df = errcode_weight_df.groupby('user_id')['errcode_weight'].sum().to_frame(name='errcode_weight')
total_sum = errcode_weight_df.errcode_weight.sum()
errcode_weight_df.errcode_weight = errcode_weight_df.errcode_weight / total_sum
errcode_weight_df

,errcode_weight
user_id,
10000,1.897493e-05
10001,1.522731e-04
10002,1.835360e-05
10003,1.807085e-05
10004,4.840085e-05
...,...
24995,1.210967e-05
24996,2.755471e-07
24997,4.983081e-05


In [184]:
# 모든 column 합쳐주기
# df1 = df1.groupby('user_id').sum()
# df1.drop(columns=['hour', 'errtype_weight', 'errcode_weight'], inplace=True)
dfs = [df1, user_has_multiple_models, user_has_multiple_fwvers, errtype_df, errtype_weight_df, errcode_weight_df, quality_df]
dfs = dfs[0].join(dfs[1:])
dfs

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,,,,,,,,,,,,,,,,,,,,,
10000,316,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
10001,2345,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,305,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,30.0,45.0,96.0,0.0,2.0,95.0,-1.0,0.0
10003,297,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,777,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,3.0,86.0,174.0,0.0,0.0,14.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,191,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.0,-2.0,3.0,-2.0,0.0,0.0,0.0,15.0,-2.0,0.0
24996,4,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,818,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,16.0,-1.0,0.0,0.0,0.0,91.0,-1.0,0.0


In [186]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 10000 to 24999
Columns: 104 entries, counts to quality_12
dtypes: float64(100), int32(2), int64(2)
memory usage: 12.4 MB


In [ ]:
train_err.drop(columns=['time', 'errcode'], inplace=True)
train_err

,user_id,model_nm,fwver,errtype
0,10000,model_3,05.15.2138,15
1,10000,model_3,05.15.2138,12
2,10000,model_3,05.15.2138,11
3,10000,model_3,05.15.2138,16
4,10000,model_3,05.15.2138,4
...,...,...,...,...
15367996,24999,model_3,05.15.2138,15
15367997,24999,model_3,05.15.2138,16
15367998,24999,model_3,05.15.2138,4
15367999,24999,model_3,05.15.2138,4


In [72]:
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [80]:
X = train_err
y = problem

print(X.shape, y.shape)

(15000, 88) (15000,)


In [79]:
from xgboost import XGBClassifier              # 1. XGBoost
from lightgbm.sklearn import LGBMClassifier    # 2. LightGBM

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [81]:
model1 = XGBClassifier()
model1.fit(X,y)
pred1 = model1.predict(X)
print(  confusion_matrix(y,pred1),  "\n")
print(  classification_report(y,pred1)  )

[[9869  131]
 [1466 3534]] 

              precision    recall  f1-score   support

         0.0       0.87      0.99      0.93     10000
         1.0       0.96      0.71      0.82      5000

    accuracy                           0.89     15000
   macro avg       0.92      0.85      0.87     15000
weighted avg       0.90      0.89      0.89     15000



In [82]:
model2 = LGBMClassifier()
model2.fit(X,y)
pred2 = model2.predict(X)
print(  confusion_matrix(y,pred2),  "\n")
print(  classification_report(y,pred2)  )

[[9701  299]
 [1956 3044]] 

              precision    recall  f1-score   support

         0.0       0.83      0.97      0.90     10000
         1.0       0.91      0.61      0.73      5000

    accuracy                           0.85     15000
   macro avg       0.87      0.79      0.81     15000
weighted avg       0.86      0.85      0.84     15000

